In [21]:
import pandas as pd
import openai

In [26]:
# API 키 지정하여 클라이언트 선언하기
OPENAI_API_KEY="API_KEY"

client = openai.OpenAI(api_key = "")

In [24]:
total = pd.read_csv("N:/개인/DA/내배캠/5. 최종 프로젝트/after_cleansing/test.csv", index_col=0)
total = total[:1]
total

,Platform,Number,Store,Address,Addr,Review_score,Review_counts,ID,Date,Review_text,Time
0,naver,1,아베베베이커리서울,"서울 종로구 청계천로 201 1, 2, 3층",서울 종로구,NaN,"2,978",sysysysysysysy,4.2.화,광장시장 들른김에 방문하게 된 아베베!!평일 웨이팅 없이 방문했어요! 너무너무 맛보...,2024-04-05 - 16:00:51


In [25]:
system_prompt = "You must:1. Analyze keywords in detail from the provided bakery reviews and assign scores for all 27 characteristics. For characteristics 1 through 25, output scores between -100 and +100. The 'Pets' keyword must be related to the 'customer's' 'pet'. Wild animals do not qualify. 2. Read the input review text and determine the 26th characteristic, the emotional index, as either 0 or 1. 1 indicates a positive evaluation, and 0 indicates a negative evaluation. If the emotional index cannot be determined, give it a 1. 3. For the 27th characteristic, 'special menu item name', output the name of the special bread mentioned in the review in the order of the characteristics. If none, output 'None'. 4. The result should be output as a list of numbers only. Output example: [40, 50, 0, 0, 0, 10, -5, 0, 10, 0, 10, 30, -70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'bread name'] Characteristics: 'delicious bread', 'delicious drinks', 'vegan', 'no senior zone', 'no kids zone', 'pets allowed', 'uses whole wheat', 'good view', 'good atmosphere', 'friendly', 'clean store', 'clean restrooms', 'parking available', 'spacious store', 'plenty of seating', 'comfortable seating', 'special menu available', 'photo', 'good music', 'reservation available', 'unique concept', 'beautiful interior', 'price', 'waiting', 'healthy', 'overall review emotional score', 'special menu item name' **Please make sure to output the results in the format shown in the example.** ** No explanation is needed.**"

In [27]:
bread = []
beverage = []
vegan = []
no_senior = []
no_kids = []
pet = []
wholewheat = []
nice_view = []
atmosphere = []
friendly = []
clean_store = []
restroom_clean = []
parking = []
spacious_store = []
plenty_seats = []
comfortable_seats = []
special_menu = []
photo = []
good_music = []
reservation_possible = []
unique_concept = []
beautiful_interior = []
price = []
wait = []
health = []
sentiment = []
bread_name = []

total_prompt_tokens = 0
total_completion_tokens = 0

def response_gpt(prompt):
  global total_prompt_tokens, total_completion_tokens
  response = client.chat.completions.create(
  model="gpt-4-turbo-2024-04-09",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}],
  temperature = 0
    )
  
  total_prompt_tokens+=response.usage.prompt_tokens
  total_completion_tokens+=response.usage.completion_tokens

  return response.choices[0].message.content

result = bread, beverage, vegan, no_senior, no_kids, pet, wholewheat, nice_view, atmosphere, friendly, clean_store, restroom_clean, parking, spacious_store, plenty_seats, comfortable_seats, special_menu, photo, good_music, reservation_possible, unique_concept, beautiful_interior, price, wait, health, sentiment, bread_name

for i in total['Review_text']:
  answer = response_gpt(i)
  answer2 = answer.replace('[', '').replace(']', '')
  answer_list = answer2.split(',')

  for i in range(len(answer_list)):
    if i < 26:
      result[i].append(answer_list[i].replace(' ',''))
    elif i == 26:
      text = [j.replace("'",'').replace(' ','') for j in answer_list[26:]]
      result[-1].append(text)

df = pd.DataFrame()
df['bread'] = bread
df['beverage'] = beverage
df['vegan'] = vegan
df['no_senior'] = no_senior
df['no_kids'] = no_kids
df['pet'] = pet
df['whole_wheat'] = wholewheat
df['view'] = nice_view
df['atmosphere'] = atmosphere
df['friendly'] = friendly
df['clean_store'] = clean_store
df['restroom_clean'] = restroom_clean
df['parking'] = parking
df['spacious_store'] = spacious_store
df['plenty_seats'] = plenty_seats
df['comfortable_seats'] = comfortable_seats
df['special_menu'] = special_menu
df['photo'] = photo
df['good_music'] = good_music
df['reservation_possible'] = reservation_possible
df['unique_concept'] = unique_concept
df['beautiful_interior'] = beautiful_interior
df['price'] = price
df['wait'] = wait
df['health'] = health
df['sentiment'] = sentiment
df['bread_name'] = bread_name
df

total_bill_USD = (total_prompt_tokens*0.01 + total_completion_tokens*0.03)/1000
# 24.04.15 기준 환율 1USD = 1383.50 KRW
total_bill_KRW = total_bill_USD*1383.50
print("총 소모 비용 : {} 원".format(total_bill_KRW))

총 소모 비용 : 12.631355 원


In [28]:
df

,bread,beverage,vegan,no_senior,no_kids,pet,whole_wheat,view,atmosphere,friendly,...,photo,good_music,reservation_possible,unique_concept,beautiful_interior,price,wait,health,sentiment,bread_name
0,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[조천오메기품은단팥빵]
